In [1]:
#단어 -> 수치화(DTM, Word2Vec 등)
#문서간 단어들의 차이를 계산? 유클리드, 코사인 유사도 등

In [2]:
# 문서1     1         1         0         1
# 문서2     1         0         1         1
# 문서3     2         0         2         2
import numpy as np
from numpy.linalg import norm

def cos_sim(x,y):
     return np.dot(x,y)/(norm(x)*norm(y))
    
doc1=np.array([1,1,0,1])
doc2=np.array([1,0,1,1])
doc3=np.array([2,0,2,2])

print(cos_sim(doc1, doc2))
print(cos_sim(doc1, doc3))
print(cos_sim(doc3, doc2))

0.6666666666666667
0.6666666666666667
1.0000000000000002


In [5]:
import pandas as pd
data=pd.read_csv("the-movies-dataset/movies_metadata.csv")
data.head()

C:\Users\user\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [6]:
data=data.head(20000)

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
#단순 코사인 유사도 기반 계산(연습)
tfidf=TfidfVectorizer(stop_words='english')
tfidf
#data['overview'].isnull().value_counts()
data['overview']=data['overview'].fillna('') 
#NaN의 경우는 ''로 (tfidf 작업시 NaN있으면 에러가 발생)
data['overview'].isnull().value_counts()

False    20000
Name: overview, dtype: int64

In [8]:
tfidf_mat=tfidf.fit_transform(data['overview'])
tfidf_mat.shape

(20000, 47487)

In [9]:
from sklearn.metrics.pairwise import linear_kernel
cos_sim=linear_kernel(tfidf_mat,tfidf_mat)

In [10]:
idx=pd.Series(data.index, index=data['title']).drop_duplicates()
idx.head()

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
dtype: int64

In [11]:
print(idx['Rambo'])
print(type(cos_sim[12356]))

12356
<class 'numpy.ndarray'>


In [12]:
def get_recommendations(title, cos_sim=cos_sim):
    idx_title=idx[title] # 12356=idx['Rambo']    
    sim_score=list(enumerate(cos_sim[idx_title]))
    sim_score=sorted(sim_score, key=lambda x:x[1] ,reverse=True)
    mi=sim_score[1:11]
    res=[i[0] for i in mi]
    print(data['title'].iloc[res])    

In [13]:
get_recommendations('Rambo')

2290                         First Blood
2291                           Rambo III
2289          Rambo: First Blood Part II
2693                   Universal Soldier
3832                          Billy Jack
9632     Strawberries in the Supermarket
1165                      Apocalypse Now
12252              Journey from the Fall
16638                     Little Soldier
10030                         The Search
Name: title, dtype: object


In [26]:
import requests
login_info={'m_id':'아이디',
           'm_passwd':'비밀번호'}
url_login="http://www.hanbit.co.kr/member/login_proc.php"

# post방식으로 서버에 연결 (개인정보를 불러올 때, 사용하는 방식 --> 아이디 비번 노출되지 않는다)
session = requests.session()
res = session.post(url_login,data=login_info)
res

url_mypage="http://www.hanbit.co.kr/myhanbit/myhanbit.html"
res = session.get(url_mypage)

from bs4 import BeautifulSoup
soup=BeautifulSoup(res.text,'html.parser')
# soup.select_one("#container > div > div.sm_mymileage > dl.mileage_section2 > dd > span").get_text()
mi = soup.select_one(".mileage_section1 span").get_text()
ec = soup.select_one("#container > div > div.sm_mymileage > dl.mileage_section2 > dd > span").get_text()
print("마일리지:"+mi,"이코인"+ec)

마일리지:2,000 이코인0
